In [ ]:
#!/usr/bin/env python
"""
CSDA Transport Equation Solver with Angular Discretization

Solves the equation:
    Ω ∂ψ(x,E,Ω)/∂x + ∂[S(E) ψ(x,E,Ω)]/∂E + Σ_t(E) ψ(x,E,Ω) = Q(x,E)
in a 2D (x, E) domain using a discrete ordinates method.
The angular variable (μ) is discretized using a 6‑point Gauss–Legendre quadrature
over μ ∈ [–1, 1] (providing both right- and left-moving directions).
The spatial domain is [0, 0.3] (20 cells) and the energy domain is represented by 50 cells,
with energy decreasing from 1.0 MeV to 0.01 MeV (continuous slowing-down).
Cross-section and stopping power data are read from an HDF5 file.
"""

import mfem.ser as mfem
import numpy as np
import h5py
import math
import matplotlib.pyplot as plt
from glvis import glvis

###############################################################################
# MeshGenerator: Create a 2D mesh in (x, E)
###############################################################################
def create_2D_mesh(nx, nE, x_start, x_end, E_start, E_end):
    """
    Create a 2D Cartesian mesh for the (x,E) domain.

    The x-coordinate spans [x_start, x_end] (nx cells).
    The second coordinate is a normalized variable in [0,1] that is mapped to energy
    via E = E_end + y*(E_start - E_end) (nE cells).

    Args:
        nx (int): Number of cells in the x-direction.
        nE (int): Number of cells in the energy direction.
        x_start (float): Starting x value.
        x_end (float): Ending x value.
        E_start (float): Highest energy value.
        E_end (float): Lowest energy value.

    Returns:
        mfem.Mesh: The generated 2D mesh.
    """
    mesh = mfem.Mesh(nx, nE, "QUADRILATERAL", True, 0.0, 0.0)
    x_coords = np.linspace(x_start, x_end, nx + 1)
    y_coords = np.linspace(0, 1, nE + 1)  # normalized energy coordinate

    verts = mesh.GetVertexArray()
    k = 0
    for j in range(nE + 1):
        for i in range(nx + 1):
            verts[k][0] = x_coords[i]
            verts[k][1] = y_coords[j]
            k += 1
    return mesh

###############################################################################
# DataReader: Read cross-section and stopping power data from HDF5
###############################################################################
def read_data(nE):
    """
    Read cross-section and stopping power data from an HDF5 file.

    Expects datasets "xs_t_{nE}", "xs_s_{nE}", and "S_{nE}".

    Args:
        nE (int): Number of energy groups.

    Returns:
        tuple: (xs_t_arr, xs_s_arr, S_arr) as numpy arrays.
    """
    data_file = "data.h5"
    with h5py.File(data_file, "r") as f:
        xs_t_arr = f[f"xs_t_{nE}"][:]
        xs_s_arr = f[f"xs_s_{nE}"][:]
        S_arr    = f[f"S_{nE}"][:]
    return xs_t_arr, xs_s_arr, S_arr

###############################################################################
# Coefficients: Define physical coefficients for the weak form
###############################################################################
class CrossSectionCoefficient(mfem.PyCoefficient):
    """
    Coefficient for the total cross-section Σ_t(E).

    Maps the normalized energy coordinate (x[1]) to the appropriate cross-section value.
    """
    def __init__(self, xs_t_data, E_start, E_end):
        super(CrossSectionCoefficient, self).__init__()
        self.xs_t_data = xs_t_data
        self.E_start = E_start
        self.E_end = E_end

    def EvalValue(self, x):
        y = x[1]  # normalized energy coordinate in [0,1]
        E = self.E_end + y * (self.E_start - self.E_end)
        n_groups = len(self.xs_t_data)
        group = min(n_groups - 1, int((E - self.E_end) / (self.E_start - self.E_end) * n_groups))
        return float(self.xs_t_data[group])

class StoppingPowerCoefficient(mfem.PyCoefficient):
    """
    Coefficient for the stopping power S(E).

    Maps the normalized energy coordinate (x[1]) to the corresponding S(E) value.
    """
    def __init__(self, S_data, E_start, E_end):
        super(StoppingPowerCoefficient, self).__init__()
        self.S_data = S_data
        self.E_start = E_start
        self.E_end = E_end

    def EvalValue(self, x):
        y = x[1]
        E = self.E_end + y * (self.E_start - self.E_end)
        n_groups = len(self.S_data)
        group = min(n_groups - 1, int((E - self.E_end) / (self.E_start - self.E_end) * n_groups))
        return float(self.S_data[group])

def Q_function(x):
    """
    Source term Q(x,E). For this example, returns zero.

    Args:
        x (mfem.Vector): The coordinate vector.

    Returns:
        float: Source term (0.0).
    """
    return 0.0

class InflowCoefficient(mfem.PyCoefficient):
    """
    Inflow boundary coefficient for the CSDA transport equation.

    This coefficient returns a unit inflow flux when the spatial coordinate x is near 0
    and the physical energy (mapped from the normalized coordinate x[1]) is near 1.0 MeV.
    Otherwise, it returns 0.0.
    """
    def __init__(self):
        super(InflowCoefficient, self).__init__()

    def EvalValue(self, x):
        """
        Evaluate the inflow boundary condition at a given point.

        Args:
            x (mfem.Vector): The coordinate vector, where x[0] is the spatial coordinate
                             and x[1] is the normalized energy coordinate.

        Returns:
            float: The inflow flux value (1.0 if at the inflow boundary, else 0.0).
        """
        E_start = 1.0
        E_end = 0.01
        y = x[1]
        # Map the normalized energy coordinate to physical energy:
        E = E_end + y * (E_start - E_end)
        # Apply the inflow condition: if x[0] is near 0 and E is near 1.0, return unit flux.
        if abs(x[0]) < 1e-3 and abs(E - 1.0) < 1e-3:
            return 1.0
        return 0.0

###############################################################################
# Angular Discretization: Get discrete ordinates from Gauss-Legendre quadrature
###############################################################################
def get_angular_directions():
    """
    Retrieve discrete angular directions (μ) and weights using a 6-point Gauss-Legendre rule.

    Returns:
        list of tuples: Each tuple is (mu, weight), where mu ∈ [-1, 1].
    """
    IR = mfem.IntRules.Get(mfem.Geometry.SEGMENT, 6)
    directions = []
    for i in range(IR.GetNPoints()):
        ip = IR.IntPoint(i)
        # ip.x is the Gauss-Legendre point in [-1, 1]
        directions.append((ip.x, ip.weight))
    return directions

###############################################################################
# Assembler: Assemble the DG weak form for one discrete angle
###############################################################################
def assemble_system_for_angle(fes, mesh, E_range, xs_t_arr, S_arr, mu, inflow_coeff):
    """
    Assemble the DG system for the CSDA transport equation for a given angular direction.
    ...
    """
    E_start, E_end = E_range

    # Create physical coefficients.
    xs_t_coeff = CrossSectionCoefficient(xs_t_arr, E_start, E_end)
    S_coeff = StoppingPowerCoefficient(S_arr, E_start, E_end)
    
    # Instead of wrapping a Python function, use a constant coefficient for Q since Q=0 everywhere.
    Q_coeff = mfem.ConstantCoefficient(0.0)
    
    # Use provided inflow_coeff (wrapped as a PyCoefficient subclass or otherwise)
    # Create a constant coefficient for the angular term (μ).
    mu_coeff = mfem.ConstantCoefficient(mu)

    # Build the bilinear form.
    a = mfem.BilinearForm(fes)
    a.AddDomainIntegrator(mfem.ConvectionIntegrator(mu_coeff, 1.0))  # spatial convection: μ ∂ψ/∂x
    a.AddDomainIntegrator(mfem.ConvectionIntegrator(S_coeff, 1.0))   # energy convection: ∂(S(E)ψ)/∂E
    a.AddDomainIntegrator(mfem.MassIntegrator(xs_t_coeff))           # absorption: Σ_t ψ
    a.AddInteriorFaceIntegrator(mfem.DGTraceIntegrator(mu_coeff, 1.0, -0.5))
    a.AddInteriorFaceIntegrator(mfem.DGTraceIntegrator(S_coeff, 1.0, -0.5))
    a.Assemble()
    A = a.SpMat()

    # Assemble the right-hand side.
    b = mfem.LinearForm(fes)
    b.AddDomainIntegrator(mfem.DomainLFIntegrator(Q_coeff))
    b.AddBoundaryIntegrator(mfem.BoundaryLFIntegrator(inflow_coeff))
    b.Assemble()

    return A, b


###############################################################################
# TransportSolver: Solve the system for a given discrete angle
###############################################################################
class TransportSolver:
    """
    Solver for the CSDA transport equation for a given discrete angle.

    Provides methods to solve using a direct LU solver.
    """
    def __init__(self, A, b, fes):
        """
        Initialize the solver.

        Args:
            A (mfem.SparseMatrix): System matrix.
            b (mfem.Vector): Right-hand side.
            fes (mfem.FiniteElementSpace): Finite element space.
        """
        self.A = A
        self.b = b
        self.fes = fes

    def solve_lu(self):
        """
        Solve the system using a direct LU solver.

        Returns:
            mfem.Vector: The solution vector.
        """
        x = mfem.Vector(self.b.Size())
        lu_solver = mfem.SuperLUSolver()
        lu_solver.SetOperator(self.A)
        lu_solver.Mult(self.b, x)
        return x

###############################################################################
# Visualizer: Plot the solution using GLVis and Matplotlib
###############################################################################
def glvis_visualize(mesh, solution, title="CSDA Transport Solution"):
    """
    Visualize the solution using GLVis.

    Args:
        mesh (mfem.Mesh): The mesh.
        solution (mfem.GridFunction): The solution.
        title (str): Title for the GLVis window.
    """
    ss = mfem.socketstream("localhost", 19916)
    if not ss.good():
        print("Unable to open GLVis socket connection.")
        return
    ss.send_text("solution\n" + title + "\n")
    mesh.Print(ss)
    solution.Save(ss)
    ss.send_text("\n")

def matplotlib_visualize(mesh, solution, E_range, nx, nE):
    """
    Visualize the solution using Matplotlib.

    Maps the normalized energy coordinate y to physical energy:
        E = E_end + y*(E_start - E_end)
    
    Args:
        mesh (mfem.Mesh): The mesh.
        solution (mfem.GridFunction): The solution.
        E_range (tuple): (E_start, E_end).
        nx (int): Number of spatial cells.
        nE (int): Number of energy cells.
    """
    nodes = mesh.GetNodes()
    if nodes.Size() == 0:
        print("Mesh has no nodes; cannot plot.")
        return
    num_nodes = nodes.Height()
    x_coords = np.array([nodes.Get(i, 0) for i in range(num_nodes)])
    y_coords = np.array([nodes.Get(i, 1) for i in range(num_nodes)])
    E_start, E_end = E_range
    E_values = E_end + y_coords * (E_start - E_end)
    sol = np.array([solution.GetValue(i) for i in range(num_nodes)])
    plt.figure()
    sc = plt.scatter(x_coords, E_values, c=sol, cmap='viridis')
    plt.xlabel('x')
    plt.ylabel('Energy (MeV)')
    plt.title('CSDA Transport Scalar Flux')
    plt.colorbar(sc, label='Flux')
    plt.show()

###############################################################################
# Main driver: Loop over discrete angles and integrate the solution
###############################################################################
def main():
    # Problem parameters
    nx = 20
    nE = 50
    x_start = 0.0
    x_end = 0.3
    E_start = 1.0   # highest energy
    E_end = 0.01    # lowest energy
    E_range = (E_start, E_end)

    # Read data from HDF5.
    xs_t_arr, xs_s_arr, S_arr = read_data(nE)

    # Create mesh.
    mesh = create_2D_mesh(nx, nE, x_start, x_end, E_start, E_end)
    
    # Define finite element space (L2, order=1).
    order = 1
    fec = mfem.L2_FECollection(order, mesh.Dimension())
    fes = mfem.FiniteElementSpace(mesh, fec)
    print("Number of unknowns:", fes.GetVSize())

    # Define inflow coefficient (wrapped as a FunctionCoefficient).
    inflow_coeff = InflowCoefficient()

    # Get discrete angular directions and weights.
    directions = get_angular_directions()  # list of (mu, weight)
    
    # Initialize a vector to accumulate the angular-integrated solution.
    # We use a GridFunction in the given FE space.
    scalar_flux = mfem.GridFunction(fes)
    scalar_flux.Assign(0.0)
    
    # Loop over each discrete angle.
    for (mu, weight) in directions:
        print("Solving for mu =", mu, "with weight =", weight)
        # Assemble the system for this angle.
        A, b = assemble_system_for_angle(fes, mesh, E_range, xs_t_arr, S_arr, mu, inflow_coeff)
        # Solve the system.
        solver = TransportSolver(A, b, fes)
        sol = solver.solve_lu()  # using LU for simplicity
        # Convert the solution vector into a GridFunction.
        sol_gf = mfem.GridFunction(fes)
        sol_gf.Assign(sol)
        # Accumulate weighted solution.
        scalar_flux.Add(weight, sol_gf)
    
    # Optionally, scale the scalar flux by the sum of weights (for normalization).
    total_weight = sum(w for (_, w) in directions)
    if total_weight > 0:
        scalar_flux.Scale(1.0 / total_weight)
    
    # Visualize the angular-integrated (scalar) flux.
    glvis_visualize(mesh, scalar_flux, "CSDA Transport Scalar Flux")
    matplotlib_visualize(mesh, scalar_flux, E_range, nx, nE)

if __name__ == "__main__":
    main()


Number of unknowns: 4000
Solving for mu = 0.06943184420297371 with weight = 0.1739274225687269


TypeError: in method 'new_ConvectionIntegrator', argument 1 of type 'mfem::VectorCoefficient &'

In [3]:
#!/usr/bin/env python
"""
CSDA Transport Equation Solver with Angular Discretization

Solves the equation:
    Ω ∂ψ(x,E)/∂x + ∂[S(E) ψ(x,E)]/∂E + Σ_t(E) ψ(x,E) = Q(x,E)
in a 2D (x, E) domain using discrete ordinates.
The angular variable (μ) is discretized using a 6‑point Gauss–Legendre rule
over μ ∈ [–1, 1] (yielding both right- and left-moving directions).
The spatial domain is [0, 0.3] (20 cells) and the energy domain is represented by 50 cells,
with energy decreasing from 1.0 MeV to 0.01 MeV.
Cross-section and stopping power data are read from an HDF5 file.
"""

import mfem.ser as mfem
import numpy as np
import h5py
import math
import matplotlib.pyplot as plt
from glvis import glvis

###############################################################################
# MeshGenerator: Create a 2D mesh in (x, E)
###############################################################################
def create_2D_mesh(nx, nE, x_start, x_end, E_start, E_end):
    """
    Create a 2D Cartesian mesh for the (x,E) domain.

    The x-coordinate spans [x_start, x_end] (nx cells).
    The second coordinate is a normalized variable in [0,1] that is mapped to energy
    via E = E_end + y*(E_start - E_end) (nE cells).

    Args:
        nx (int): Number of cells in x-direction.
        nE (int): Number of cells in energy direction.
        x_start (float): Starting x value.
        x_end (float): Ending x value.
        E_start (float): Highest energy value.
        E_end (float): Lowest energy value.

    Returns:
        mfem.Mesh: Generated 2D mesh.
    """
    mesh = mfem.Mesh(nx, nE, "QUADRILATERAL", True, 0.0, 0.0)
    x_coords = np.linspace(x_start, x_end, nx + 1)
    y_coords = np.linspace(0, 1, nE + 1)  # normalized energy coordinate

    verts = mesh.GetVertexArray()
    k = 0
    for j in range(nE + 1):
        for i in range(nx + 1):
            verts[k][0] = x_coords[i]
            verts[k][1] = y_coords[j]
            k += 1
    return mesh

###############################################################################
# DataReader: Read cross-section and stopping power data from HDF5
###############################################################################
def read_data(nE):
    """
    Read cross-section and stopping power data from an HDF5 file.

    Expects datasets "xs_t_{nE}", "xs_s_{nE}", and "S_{nE}".

    Args:
        nE (int): Number of energy groups.

    Returns:
        tuple: (xs_t_arr, xs_s_arr, S_arr) as numpy arrays.
    """
    data_file = "data.h5"
    with h5py.File(data_file, "r") as f:
        xs_t_arr = f[f"xs_t_{nE}"][:]
        xs_s_arr = f[f"xs_s_{nE}"][:]
        S_arr    = f[f"S_{nE}"][:]
    return xs_t_arr, xs_s_arr, S_arr

###############################################################################
# Coefficient Wrappers
###############################################################################
class CrossSectionCoefficient(mfem.PyCoefficient):
    """
    Coefficient for the total cross-section Σ_t(E).

    Maps the normalized energy coordinate (x[1]) to the appropriate cross-section value.
    """
    def __init__(self, xs_t_data, E_start, E_end):
        super(CrossSectionCoefficient, self).__init__()
        self.xs_t_data = xs_t_data
        self.E_start = E_start
        self.E_end = E_end

    def EvalValue(self, x):
        y = x[1]  # normalized energy coordinate in [0,1]
        E = self.E_end + y * (self.E_start - self.E_end)
        n_groups = len(self.xs_t_data)
        group = min(n_groups - 1, int((E - self.E_end) / (self.E_start - self.E_end) * n_groups))
        return float(self.xs_t_data[group])

class StoppingPowerCoefficient(mfem.PyCoefficient):
    """
    Coefficient for the stopping power S(E).

    Maps the normalized energy coordinate (x[1]) to the corresponding S(E) value.
    """
    def __init__(self, S_data, E_start, E_end):
        super(StoppingPowerCoefficient, self).__init__()
        self.S_data = S_data
        self.E_start = E_start
        self.E_end = E_end

    def EvalValue(self, x):
        y = x[1]
        E = self.E_end + y * (self.E_start - self.E_end)
        n_groups = len(self.S_data)
        group = min(n_groups - 1, int((E - self.E_end) / (self.E_start - self.E_end) * n_groups))
        return float(self.S_data[group])

def Q_function(x):
    """
    Source term Q(x,E). Returns 0.0 in this example.

    Args:
        x (mfem.Vector): The coordinate vector.
        
    Returns:
        float: 0.0.
    """
    return 0.0

def inflow_function(x):
    """
    Inflow boundary condition function.

    Imposes unit flux at the spatial boundary (x=0) when the physical energy is near 1.0 MeV.

    Args:
        x (mfem.Vector): The coordinate vector.
        
    Returns:
        float: Inflow flux value.
    """
    E_start = 1.0
    E_end = 0.01
    y = x[1]
    E = E_end + y * (E_start - E_end)
    if abs(x[0]) < 1e-3 and abs(E - 1.0) < 1e-3:
        return 1.0
    return 0.0

###############################################################################
# Vector Coefficient Wrappers for Convection Integrators
###############################################################################
class AngularVectorCoefficient(mfem.VectorPyCoefficient):
    """
    A vector coefficient for the angular term.

    Returns a constant vector [mu, 0] representing the angular direction
    in the (x,E) space.
    """
    def __init__(self, mu, dim=2):
        super(AngularVectorCoefficient, self).__init__(dim)
        self.mu = mu
        self.dim = dim

    def EvalValue(self, x):
        """
        Evaluate the angular vector coefficient at a point x.

        Args:
            x (mfem.Vector): The coordinate vector.
        
        Returns:
            list: A list representing the vector [mu, 0] in 2D (or [mu, 0, 0] in 3D).
        """
        if self.dim == 1:
            return [self.mu]
        elif self.dim == 2:
            return [self.mu, 0.0]
        elif self.dim == 3:
            return [self.mu, 0.0, 0.0]

class EnergyVectorCoefficient(mfem.VectorPyCoefficient):
    """
    A vector coefficient that wraps a scalar coefficient for energy convection.

    Given a scalar coefficient (for example, stopping power S(E)),
    returns a vector [0, S(E)] in 2D.
    """
    def __init__(self, scalar_coeff, dim=2):
        super(EnergyVectorCoefficient, self).__init__(dim)
        self.scalar_coeff = scalar_coeff
        self.dim = dim

    def EvalValue(self, x):
        """
        Evaluate the energy vector coefficient at a point x.

        Args:
            x (mfem.Vector): The coordinate vector.
        
        Returns:
            list: A list representing the vector [0, S(E)] in 2D (or [0, S(E), 0] in 3D).
        """
        S_val = self.scalar_coeff.EvalValue(x)
        if self.dim == 1:
            return [S_val]
        elif self.dim == 2:
            return [0.0, S_val]
        elif self.dim == 3:
            return [0.0, S_val, 0.0]


###############################################################################
# Assembler: Assemble the DG weak form for one discrete angle
###############################################################################
def assemble_system_for_angle(fes, mesh, E_range, xs_t_arr, S_arr, mu, inflow_coeff):
    """
    Assemble the DG system for the CSDA transport equation for a given angular direction.

    For a fixed discrete angle μ, the weak form is:
        μ ∂ψ/∂x + ∂(S(E)ψ)/∂E + Σ_t(E)ψ = Q(x,E)
    with DG interface integrals for upwinding.

    Args:
        fes (mfem.FiniteElementSpace): Finite element space.
        mesh (mfem.Mesh): The computational mesh.
        E_range (tuple): (E_start, E_end) for energy.
        xs_t_arr (numpy.array): Total cross-section data.
        S_arr (numpy.array): Stopping power data.
        mu (float): Discrete angular cosine.
        inflow_coeff (mfem.Coefficient): Inflow boundary coefficient.

    Returns:
        tuple: (A, b) where A is the system matrix and b is the RHS vector.
    """
    E_start, E_end = E_range

    # Create physical scalar coefficients.
    xs_t_coeff = CrossSectionCoefficient(xs_t_arr, E_start, E_end)
    S_scalar_coeff = StoppingPowerCoefficient(S_arr, E_start, E_end)
    # Use constant coefficients for Q (source = 0) and for inflow, use the provided inflow_coeff.
    Q_coeff = mfem.ConstantCoefficient(0.0)

    # Create vector coefficients for convection integrators.
    # For the spatial convection term, wrap mu in a vector [mu, 0].
    mu_vector_coeff = AngularVectorCoefficient(mu, dim=2)
    # For the energy convection term, wrap the scalar S(E) into a vector [0, S(E)].
    S_vector_coeff = EnergyVectorCoefficient(S_scalar_coeff, dim=2)

    # Build the bilinear form.
    a = mfem.BilinearForm(fes)
    # Spatial convection term: μ ∂ψ/∂x.
    a.AddDomainIntegrator(mfem.ConvectionIntegrator(mu_vector_coeff, 1.0))
    # Energy convection term: ∂(S(E)ψ)/∂E.
    a.AddDomainIntegrator(mfem.ConvectionIntegrator(S_vector_coeff, 1.0))
    # Absorption term: Σ_t(E)ψ.
    a.AddDomainIntegrator(mfem.MassIntegrator(xs_t_coeff))
    # Add DG interior face integrators for upwind flux.
    a.AddInteriorFaceIntegrator(mfem.DGTraceIntegrator(mu_vector_coeff, 1.0, -0.5))
    a.AddInteriorFaceIntegrator(mfem.DGTraceIntegrator(S_vector_coeff, 1.0, -0.5))
    a.Assemble()
    A = a.SpMat()

    # Assemble the right-hand side.
    b = mfem.LinearForm(fes)
    b.AddDomainIntegrator(mfem.DomainLFIntegrator(Q_coeff))
    b.AddBoundaryIntegrator(mfem.BoundaryLFIntegrator(inflow_coeff))
    b.Assemble()

    return A, b

###############################################################################
# TransportSolver: Solve the system for a given discrete angle
###############################################################################
class TransportSolver:
    """
    Solver for the CSDA transport equation for a given discrete angle.

    Provides a direct LU solver.
    """
    def __init__(self, A, b, fes):
        """
        Initialize the solver.

        Args:
            A (mfem.SparseMatrix): The system matrix.
            b (mfem.Vector): The right-hand side.
            fes (mfem.FiniteElementSpace): The finite element space.
        """
        self.A = A
        self.b = b
        self.fes = fes

    def solve_lu(self):
        """
        Solve the system using a direct LU solver.

        Returns:
            mfem.Vector: The solution vector.
        """
        x = mfem.Vector(self.b.Size())
        lu_solver = mfem.SuperLUSolver()
        lu_solver.SetOperator(self.A)
        lu_solver.Mult(self.b, x)
        return x

###############################################################################
# Angular Discretization: Get discrete angles from Gauss-Legendre quadrature
###############################################################################
def get_angular_directions():
    """
    Retrieve discrete angular directions (μ) and weights using a 6-point Gauss-Legendre rule.

    Returns:
        list of tuples: Each tuple is (mu, weight) with μ in [-1, 1].
    """
    IR = mfem.IntRules.Get(mfem.Geometry.SEGMENT, 6)
    directions = []
    for i in range(IR.GetNPoints()):
        ip = IR.IntPoint(i)
        directions.append((ip.x, ip.weight))
    return directions

###############################################################################
# Visualizer: Plot the solution using GLVis and Matplotlib
###############################################################################
def glvis_visualize(mesh, solution, title="CSDA Transport Scalar Flux"):
    """
    Visualize the solution using GLVis.

    Args:
        mesh (mfem.Mesh): The mesh.
        solution (mfem.GridFunction): The solution.
        title (str): Title for the GLVis window.
    """
    ss = mfem.socketstream("localhost", 19916)
    if not ss.good():
        print("Unable to open GLVis socket connection.")
        return
    ss.send_text("solution\n" + title + "\n")
    mesh.Print(ss)
    solution.Save(ss)
    ss.send_text("\n")

def matplotlib_visualize(mesh, solution, E_range, nx, nE):
    """
    Visualize the solution using Matplotlib.

    Maps the normalized energy coordinate y to physical energy:
      E = E_end + y*(E_start - E_end)
    
    Args:
        mesh (mfem.Mesh): The mesh.
        solution (mfem.GridFunction): The solution.
        E_range (tuple): (E_start, E_end).
        nx (int): Number of spatial cells.
        nE (int): Number of energy cells.
    """
    nodes = mesh.GetNodes()
    if nodes.Size() == 0:
        print("Mesh has no nodes; cannot plot.")
        return
    num_nodes = nodes.Height()
    x_coords = np.array([nodes.Get(i, 0) for i in range(num_nodes)])
    y_coords = np.array([nodes.Get(i, 1) for i in range(num_nodes)])
    E_start, E_end = E_range
    E_values = E_end + y_coords * (E_start - E_end)
    sol = np.array([solution.GetValue(i) for i in range(num_nodes)])
    plt.figure()
    sc = plt.scatter(x_coords, E_values, c=sol, cmap='viridis')
    plt.xlabel('x')
    plt.ylabel('Energy (MeV)')
    plt.title('CSDA Transport Scalar Flux')
    plt.colorbar(sc, label='Flux')
    plt.show()

###############################################################################
# Main driver: Loop over discrete angles, solve, and integrate the solution
###############################################################################
def main():
    # Problem parameters
    nx = 20
    nE = 50
    x_start = 0.0
    x_end = 0.3
    E_start = 1.0   # highest energy
    E_end = 0.01    # lowest energy
    E_range = (E_start, E_end)

    # Read data from HDF5.
    xs_t_arr, xs_s_arr, S_arr = read_data(nE)

    # Create mesh.
    mesh = create_2D_mesh(nx, nE, x_start, x_end, E_start, E_end)
    
    # Define finite element space (L2, order=1).
    order = 1
    fec = mfem.L2_FECollection(order, mesh.Dimension())
    fes = mfem.FiniteElementSpace(mesh, fec)
    print("Number of unknowns:", fes.GetVSize())

    # Define inflow coefficient using a PyCoefficient subclass.
    class InflowCoefficient(mfem.PyCoefficient):
        """
        Inflow boundary coefficient that returns 1.0 when x[0] ≈ 0 and E ≈ 1.0 MeV.
        """
        def __init__(self):
            super(InflowCoefficient, self).__init__()

        def EvalValue(self, x):
            E_start = 1.0
            E_end = 0.01
            y = x[1]
            E = E_end + y * (E_start - E_end)
            if abs(x[0]) < 1e-3 and abs(E - 1.0) < 1e-3:
                return 1.0
            return 0.0

    inflow_coeff = InflowCoefficient()

    # Get discrete angular directions (μ) and weights.
    directions = get_angular_directions()  # list of (mu, weight)
    
    # Initialize a GridFunction to accumulate the angular-integrated (scalar) flux.
    scalar_flux = mfem.GridFunction(fes)
    scalar_flux.Assign(0.0)
    
    # Loop over each discrete angle.
    for (mu, weight) in directions:
        print("Solving for mu =", mu, "with weight =", weight)
        # Assemble the system for this angle.
        A, b = assemble_system_for_angle(fes, mesh, E_range, xs_t_arr, S_arr, mu, inflow_coeff)
        # Solve the system using direct LU.
        solver = TransportSolver(A, b, fes)
        sol = solver.solve_lu()
        # Convert the solution vector to a GridFunction.
        sol_gf = mfem.GridFunction(fes)
        sol_gf.Assign(sol)
        # Accumulate the weighted solution.
        scalar_flux.Add(weight, sol_gf)
    
    # Optionally normalize the scalar flux by the total weight.
    total_weight = sum(w for (_, w) in directions)
    if total_weight > 0:
        scalar_flux.Scale(1.0 / total_weight)
    
    # Visualize the angular-integrated (scalar) flux.
    glvis_visualize(mesh, scalar_flux, "CSDA Transport Scalar Flux")
    matplotlib_visualize(mesh, scalar_flux, E_range, nx, nE)

if __name__ == "__main__":
    main()


: 

In [18]:
import mfem.ser as mfem

class CrossSectionCoefficient(mfem.PyCoefficient):
    """
    Coefficient for the total cross-section Σ_t(E).

    Maps a normalized energy coordinate (x[1] ∈ [0,1]) to a physical energy E and then
    selects the corresponding group from the provided xs_t_data array.
    
    Given:
      - xs_t_data: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
      - E_start: 1.0 (highest energy)
      - E_end:   0.01 (lowest energy)
    
    The mapping from normalized coordinate y to physical energy E is:
    
        E = E_end + y * (E_start - E_end)
        
    For example:
      - If y = 0: 
          E = 0.01 + 0 * (1.0 - 0.01) = 0.01
      - If y = 1:
          E = 0.01 + 1 * (1.0 - 0.01) = 1.0
      - If y = 0.5:
          E = 0.01 + 0.5 * 0.99 = 0.01 + 0.495 = 0.505
      
    Then the group index is computed by:
    
        group = min(n_groups - 1, int((E - E_end) / (E_start - E_end) * n_groups))
    
    where n_groups = len(xs_t_data) = 10.
    
    Let's see some examples:
    
    1. For y = 0 (E = 0.01):
         (E - E_end) = (0.01 - 0.01) = 0
         0 / 0.99 = 0.0
         0.0 * 10 = 0.0  → int(0.0) = 0
         group = min(9, 0) = 0 
         ⇒ Returns xs_t_data[0] = 1.
    
    2. For y = 0.5 (E = 0.505):
         (E - E_end) = (0.505 - 0.01) = 0.495
         0.495 / 0.99 = 0.5
         0.5 * 10 = 5.0  → int(5.0) = 5
         group = min(9, 5) = 5 
         ⇒ Returns xs_t_data[5] = 6.
    
    3. For y = 1 (E = 1.0):
         (E - E_end) = (1.0 - 0.01) = 0.99
         0.99 / 0.99 = 1.0
         1.0 * 10 = 10.0 → int(10.0) = 10
         group = min(9, 10) = 9  (since we cap it at 9)
         ⇒ Returns xs_t_data[9] = 10.
    """
    def __init__(self, xs_t_data, E_start, E_end):
        super(CrossSectionCoefficient, self).__init__()
        self.xs_t_data = xs_t_data
        self.E_start = E_start
        self.E_end = E_end

    def EvalValue(self, x):
        # x[1] is the normalized energy coordinate, y ∈ [0,1]
        y = x[1]
        # Map y to physical energy E:
        #   E = E_end + y*(E_start - E_end)
        E = self.E_end + y * (self.E_start - self.E_end)
        
        # Number of groups (e.g., if xs_t_data = [1,2,...,10], n_groups = 10)
        n_groups = len(self.xs_t_data)
        
        # Compute group index:
        #   group = min(n_groups - 1, int((E - E_end) / (E_start - E_end) * n_groups))
        group = min(n_groups - 1, int((E - self.E_end) / (self.E_start - self.E_end) * n_groups))
        
        # Debug print (if needed, you can uncomment below)
        # print("y = {:.3f}, E = {:.3f}, group = {}".format(y, E, group))
        
        return float(self.xs_t_data[group])

# Example Data
xs_t_data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
E_start = 1.0
E_end = 0.00

# For testing, we simulate evaluation at several normalized energy values:
import mfem.ser as mfem
import numpy as np

# Create a dummy mfem.Vector for testing; assume it has at least 2 entries.
def test_coefficient(y_value):
    x = mfem.Vector(2)
    # x[0] can be arbitrary (not used in EvalValue)
    x[0] = 0.0
    x[1] = y_value
    coeff = CrossSectionCoefficient(xs_t_data, E_start, E_end)
    return coeff.EvalValue(x)

# Test at several points:
for y_val in [0.0, 0.75, 0.85, 0.9, 0.95,  1.0]:
    value = test_coefficient(y_val)
    # For clarity, also compute E and group manually:
    E = E_end + y_val * (E_start - E_end)
    group = min(len(xs_t_data) - 1, int((E - E_end) / (E_start - E_end) * len(xs_t_data)))
    print("For normalized y = {:.2f}:".format(y_val))
    print("  Computed physical energy E = {:.3f}".format(E))
    print("  Energy group index = {}".format(group))
    print("  Returned xs_t value = {}".format(value))
    print()


For normalized y = 0.00:
  Computed physical energy E = 0.000
  Energy group index = 0
  Returned xs_t value = 1.0

For normalized y = 0.75:
  Computed physical energy E = 0.750
  Energy group index = 7
  Returned xs_t value = 8.0

For normalized y = 0.85:
  Computed physical energy E = 0.850
  Energy group index = 8
  Returned xs_t value = 9.0

For normalized y = 0.90:
  Computed physical energy E = 0.900
  Energy group index = 9
  Returned xs_t value = 10.0

For normalized y = 0.95:
  Computed physical energy E = 0.950
  Energy group index = 9
  Returned xs_t value = 10.0

For normalized y = 1.00:
  Computed physical energy E = 1.000
  Energy group index = 9
  Returned xs_t value = 10.0



In [ ]:
#!/usr/bin/env python
"""
Bu örnek, particle transport denklemini aşağıdaki formda çözer:

    μ ∂ψ/∂x + ∂(S(E) ψ)/∂E + [S'(E)+Σ_t(E)] ψ = Q(x,E)

Burada:
 - μ: Açısal değişken; Gauss–Legendre kuadratürü ile ayrılacak.
 - S(E) ve Σ_t(E): Enerjiye bağlı stopping power ve toplam kesit,
   tablo verileri ile belirlenecek.
 - Q(x,E): Kaynak terimi, Σ_s(E) tablosundan belirlenecek.
 
Not: Gerçek uygulamada, DG yönteminde upwind numerik akımın uygulanması gerekir.
      Bu örnekte, her discrete ordinata için basit domain integratorleri kullanılmıştır.
"""

import mfem.ser as mfem
import numpy as np
import sys

# Problem ve mesh parametreleri
L = 1.0                 # x-domain uzunluğu
E_min = 0.01            # Enerji alt sınırı
E_max = 2.0             # Enerji üst sınırı
Nx = 50                 # x yönünde eleman sayısı
NE = 50                 # E yönünde eleman sayısı
order = 1               # FE eleman polinom derecesi
# Discrete ordinates için Gauss-Legendre noktaları (örn. 4 nokta)
nmu = 4

# Tablo verileri (örnek; gerçek veriler farklı olabilir)
E_table = np.linspace(E_min, E_max, 10)
S_table = 0.1 * E_table         # S(E) örneğin lineer
Sigma_t_table = 0.05 * E_table    # Σ_t(E)
Sigma_s_table = 0.02 * E_table    # Σ_s(E)
# S'(E) örneğinde türev sabit (0.1) olsun.
def S_prime(E):
    return 0.1

# Fonksiyon tanımları (tablo verilerini kullanarak)
def S_func(x):
    # x[1] enerji koordinatıdır
    E = x[1]
    return np.interp(E, E_table, S_table)
S_coef = mfem.FunctionCoefficient(S_func)

def Sigma_t_func(x):
    E = x[1]
    return np.interp(E, E_table, Sigma_t_table)
Sigma_t_coef = mfem.FunctionCoefficient(Sigma_t_func)

def R_func(x):
    # R(E) = S'(E) + Σ_t(E)
    E = x[1]
    return S_prime(E) + np.interp(E, E_table, Sigma_t_table)
R_coef = mfem.FunctionCoefficient(R_func)

def Q_func(x):
    # Q(x,E) = Σ_s(E) tablosundan
    E = x[1]
    return np.interp(E, E_table, Sigma_s_table)
Q_coef = mfem.FunctionCoefficient(Q_func)

# Gauss-Legendre kuadratürü ile discrete ordinates elde et
mu_vals, mu_weights = np.polynomial.legendre.leggauss(nmu)
# Burada mu_vals aralık [-1,1]'de verilir.

# Tüm discrete ordinata sonuçlarını saklamak için liste
psi_mu_list = []  # her eleman: (mu, weight, GridFunction)

# Döngü: Her discrete ordinata için sistemi kur ve çöz
for mu, w in zip(mu_vals, mu_weights):
    print("Çözüm için μ =", mu)
    # Mesh oluştur: [0,L] x [E_min,E_max]
    mesh = mfem.Mesh(Nx, NE, "TRIANGLE")
    for i in range(mesh.GetNV()):
        v = mesh.GetVertex(i)
        # x: [0,1] -> [0,L]
        v[0] = v[0] * L
        # E: [0,1] -> [E_min, E_max]
        v[1] = E_min + v[1] * (E_max - E_min)
        mesh.SetVertex(i, v)
    
    # DG için L2FE koleksiyonu (kopuk)
    fec = mfem.L2_FECollection(order, mesh.Dimension())
    fespace = mfem.FiniteElementSpace(mesh, fec)
    print("  Unknowns:", fespace.GetTrueVSize())
    
    # Kenar koşulları: Burada örneğin, her kenarda giriş koşulu verilebilir.
    # Örneğin, sağa gidenler için sol sınırdan, sola gidenler için sağ sınırdan giriş.
    # Basitlik açısından burada tüm kenarlarda homojen koşul (ψ=0) varsayalım.
    boundary_dofs = mfem.intArray()
    fespace.GetBoundaryTrueDofs(boundary_dofs)
    
    # Tanımlanacak vektör katsayısı:
    # v(x) = [μ, S(E)]
    def vel_func(x, mu_val=mu):
        # x[1] enerji
        return [mu_val, S_coef.Eval(x)]
    vel_coef = mfem.VectorFunctionCoefficient(2, vel_func)
    
    # Bilinear form: a(ψ,v) = ∫ ( v·∇ψ + (S'(E)+Σ_t(E))ψ ) v dx dE
    a = mfem.BilinearForm(fespace)
    a.AddDomainIntegrator(mfem.ConvectionIntegrator(vel_coef))
    a.AddDomainIntegrator(mfem.MassIntegrator(R_coef))
    a.Assemble()
    
    # Linear form: b(v) = ∫ Q(x,E) v dx dE
    b = mfem.LinearForm(fespace)
    b.AddDomainIntegrator(mfem.DomainLFIntegrator(Q_coef))
    b.Assemble()
    
    # Çözüm grid fonksiyonu
    psi = mfem.GridFunction(fespace)
    psi.Assign(0.0)
    
    # Sistemi derle (kenar koşullarını uygulayarak)
    A = mfem.SparseMatrix()
    B = mfem.Vector()
    X = mfem.Vector()
    a.FormLinearSystem(boundary_dofs, psi, b, A, X, B)
    
    # PCG ile çöz
    M_prec = mfem.GSSmoother(A)
    pcg = mfem.PCGSolver()
    pcg.SetOperator(A)
    pcg.SetPreconditioner(M_prec)
    pcg.SetRelTol(1e-8)
    pcg.SetMaxIter(500)
    pcg.Mult(B, X)
    
    a.RecoverFEMSolution(X, b, psi)
    
    # Sonucu listeye ekle (her discrete ordinate için)
    psi_mu_list.append( (mu, w, psi.Clone()) )
    
    # Opsiyonel: Her çözümü ayrı dosyaya kaydedebilirsin
    psi.Save("psi_mu_{:.3f}.gf".format(mu))
    mesh.Save("mesh_mu_{:.3f}.mesh".format(mu))
    
# --- Sonuçların Birleştirilmesi ---
# Örneğin, skaler (açısız) akı: ψ(x,E) = Σ_μ (w_μ * ψ_μ(x,E))
psi_scalar = psi_mu_list[0][2].Clone()  # aynı FE space, initialize to zero
psi_scalar.Assign(0.0)
for (mu, w, psi_mu) in psi_mu_list:
    psi_scalar.Add(w, psi_mu)

# Çıktı dosyasına kaydet
psi_scalar.Save("psi_scalar.gf")
print("Scalar flux dosyaya kaydedildi: psi_scalar.gf")

# Opsiyonel: Sağ ve sola giden akıları ayrı ayrı değerlendirmek için,
# mu>0 ve mu<0 listelerini oluşturabilirsin.
left_flux = None
right_flux = None
for (mu, w, psi_mu) in psi_mu_list:
    if mu < 0:
        if left_flux is None:
            left_flux = psi_mu.Clone()
            left_flux.Assign(0.0)
        left_flux.Add(w, psi_mu)
    else:
        if right_flux is None:
            right_flux = psi_mu.Clone()
            right_flux.Assign(0.0)
        right_flux.Add(w, psi_mu)
if left_flux:
    left_flux.Save("psi_left.gf")
    print("Sola giden flux: psi_left.gf")
if right_flux:
    right_flux.Save("psi_right.gf")
    print("Sağa giden flux: psi_right.gf")


In [1]:
#!/usr/bin/env python
"""
Bu örnek, aşağıdaki denklemi çözmek üzere PyMFEM kullanır:

    Ω ∂ψ/∂x + ∂(S(E) ψ)/∂E + Σ_t(E) ψ = Q(x,E)

Denklemde, ∂(S(E)ψ)/∂E ifadesi S(E)∂ψ/∂E + S'(E)ψ olarak ayrılabilir.
Bu örnekte, reaksiyon katsayısı R(E) = S'(E) + Σ_t(E) olarak kullanılacaktır.
"""

import mfem.ser as mfem
import numpy as np
import sys

# Parametreler
L = 1.0                # x-domain uzunluğu
E_min = 0.01           # Enerji alt sınırı
E_max = 2.0            # Enerji üst sınırı
Nx = 50                # x yönünde eleman sayısı
NE = 50                # E yönünde eleman sayısı
order = 2              # Finite element polinom derecesi
Omega = 1.0            # Sabit açısal yön (monodireksiyonel)

# 1. Mesh oluşturma: [0,L] x [E_min,E_max] dikdörtgeni
# mfem.Mesh(Nx, NE, "TRIANGLE") [0,1]x[0,1] mesh üretir; ikinci koordinatı ölçekleyelim.
# Mesh oluşturulduktan sonra, vertex koordinatlarını numpy array olarak al:
verts = mesh.GetVertexArray()  # verts: numpy array, shape (mesh.GetNV(), mesh.Dimension())

print("Vertex array type:", type(verts))

# x koordinatını [0,L]'ye, E koordinatını [E_min,E_max]'ye ölçekle:
verts[:, 0] *= L
verts[:, 1] = E_min + verts[:, 1] * (E_max - E_min)

# Güncellenmiş koordinatları mesh’e aktar:
mesh.SetVertexArray(verts)


# 2. FE alanı tanımlama (H1 sürekli elemanlar)
fec = mfem.H1_FECollection(order, mesh.Dimension())
fespace = mfem.FiniteElementSpace(mesh, fec)
print("Number of unknowns:", fespace.GetTrueVSize())

# 3. Kenar koşulları: Tüm kenarlarda homojen Dirichlet (ψ = 0)
boundary_dofs = mfem.intArray()
fespace.GetBoundaryTrueDofs(boundary_dofs)

# 4. Katsayı Fonksiyonları
# S(E): Enerjiye bağlı stopping power (örnek: S(E) = 0.1 * E)
def S_func(x):
    # x[0]: x, x[1]: E
    return 0.1 * x[1]
S_coef = mfem.FunctionCoefficient(S_func)

# S'(E): S fonksiyonunun türevi (örnek: türev = 0.1)
def dS_dE_func(x):
    return 0.1
dS_dE_coef = mfem.FunctionCoefficient(dS_dE_func)

# Σ_t(E): Toplam kesit (örnek: Σ_t(E) = 0.05 * E)
def Sigma_t_func(x):
    return 0.05 * x[1]
Sigma_t_coef = mfem.FunctionCoefficient(Sigma_t_func)

# Q(x,E): Kaynak terimi (örnek: sabit 1.0)
def Q_func(x):
    return 1.0
Q_coef = mfem.FunctionCoefficient(Q_func)

# Reaksiyon katsayısı: R(E) = S'(E) + Σ_t(E)
def R_func(x):
    return dS_dE_coef.Eval(x) + Sigma_t_coef.Eval(x)
R_coef = mfem.FunctionCoefficient(R_func)

# 5. Convection integrator için vektör katsayıları tanımla
# (a) x yönü: hızı (Ω, 0)
def vel1_func(x):
    return [Omega, 0.0]
vel1_coef = mfem.VectorFunctionCoefficient(2, vel1_func)

# (b) E yönü: hızı (0, S(E))
def vel2_func(x):
    return [0.0, S_coef.Eval(x)]
vel2_coef = mfem.VectorFunctionCoefficient(2, vel2_func)

# 6. Bilinear form (a) oluşturulması
a = mfem.BilinearForm(fespace)
# x yönü için adveksiyon: ∫ (Ω ∂ψ/∂x) v dx dE
a.AddDomainIntegrator(mfem.ConvectionIntegrator(vel1_coef))
# E yönü için adveksiyon: ∫ (S(E) ∂ψ/∂E) v dx dE
a.AddDomainIntegrator(mfem.ConvectionIntegrator(vel2_coef))
# Reaksiyon terimi: ∫ (S'(E) + Σ_t(E)) ψ v dx dE
a.AddDomainIntegrator(mfem.ReactionIntegrator(R_coef))
a.Assemble()

# 7. Sağ taraf (linear form) tanımlaması: ∫ Q(x,E) v dx dE
b = mfem.LinearForm(fespace)
b.AddDomainIntegrator(mfem.DomainLFIntegrator(Q_coef))
b.Assemble()

# 8. Çözüm grid fonksiyonu oluştur ve başlangıç tahmini (sıfır) ata
x_sol = mfem.GridFunction(fespace)
x_sol.Assign(0.0)

# 9. Sistemi derle: A X = B (kenar koşullarını uygulayarak)
A = mfem.SparseMatrix()
B = mfem.Vector()
X = mfem.Vector()
a.FormLinearSystem(boundary_dofs, x_sol, b, A, X, B)

# 10. Sistemi PCG ile çöz (Gauss-Seidel preconditioner örneği)
M_prec = mfem.GSSmoother(A)
pcg = mfem.PCGSolver()
pcg.SetOperator(A)
pcg.SetPreconditioner(M_prec)
pcg.SetRelTol(1e-8)
pcg.SetMaxIter(500)
pcg.Mult(B, X)

# 11. Çözümü geri al (Recover)
a.RecoverFEMSolution(X, b, x_sol)

# 12. Sonuçları dosyaya kaydet (GLVis veya benzeri ile görüntülemek için)
x_sol.Save("sol.gf")
mesh.Save("mesh.mesh")
print("Çözüm dosyalara kaydedildi: sol.gf ve mesh.mesh")


NameError: name 'mesh' is not defined

In [1]:
# Obtain discrete ordinates using Gauss-Legendre quadrature
mu_vals, mu_weights = np.polynomial.legendre.leggauss(nmu)
# Here, mu_vals are given in the interval [-1,1].

# List to store all discrete ordinate results
psi_mu_list = []  # each element: (mu, weight, GridFunction)

# Loop: Set up and solve the system for each discrete ordinate
for mu, w in zip(mu_vals, mu_weights):
    print("Solving for μ =", mu)
    # Create mesh: [0, L] x [E_min, E_max]
    mesh = mfem.Mesh(Nx, NE, "TRIANGLE")
    for i in range(mesh.GetNV()):
        v = mesh.GetVertex(i)
        # x: [0,1] -> [0,L]
        v[0] = v[0] * L
        # E: [0,1] -> [E_min, E_max]
        v[1] = E_min + v[1] * (E_max - E_min)
        mesh.SetVertex(i, v)
    
    # L2 finite element collection for DG (discontinuous Galerkin)
    fec = mfem.L2_FECollection(order, mesh.Dimension())
    fespace = mfem.FiniteElementSpace(mesh, fec)
    print("  Unknowns:", fespace.GetTrueVSize())
    
    # Boundary conditions:
    # For example, you can impose inflow conditions on each boundary.
    # For instance, for right-going particles, inflow from the left boundary,
    # and for left-going particles, inflow from the right boundary.
    # For simplicity, here we assume homogeneous conditions (ψ=0) on all boundaries.
    boundary_dofs = mfem.intArray()
    fespace.GetBoundaryTrueDofs(boundary_dofs)
    
    # Define the vector coefficient:
    # v(x) = [μ, S(E)]
    def vel_func(x, mu_val=mu):
        # x[1] is the energy
        return [mu_val, S_coef.Eval(x)]
    vel_coef = mfem.VectorFunctionCoefficient(2, vel_func)
    
    # Bilinear form: a(ψ,v) = ∫ ( v·∇ψ + (S'(E)+Σ_t(E))ψ ) v dx dE
    a = mfem.BilinearForm(fespace)
    a.AddDomainIntegrator(mfem.ConvectionIntegrator(vel_coef))
    a.AddDomainIntegrator(mfem.MassIntegrator(R_coef))
    a.Assemble()
    
    # Linear form: b(v) = ∫ Q(x,E) v dx dE
    b = mfem.LinearForm(fespace)
    b.AddDomainIntegrator(mfem.DomainLFIntegrator(Q_coef))
    b.Assemble()
    
    # Solution grid function
    psi = mfem.GridFunction(fespace)
    psi.Assign(0.0)
    
    # Assemble the system (applying boundary conditions)
    A = mfem.SparseMatrix()
    B = mfem.Vector()
    X = mfem.Vector()
    a.FormLinearSystem(boundary_dofs, psi, b, A, X, B)
    
    # Solve using PCG (Preconditioned Conjugate Gradient)
    M_prec = mfem.GSSmoother(A)
    pcg = mfem.PCGSolver()
    pcg.SetOperator(A)
    pcg.SetPreconditioner(M_prec)
    pcg.SetRelTol(1e-8)
    pcg.SetMaxIter(500)
    pcg.Mult(B, X)
    
    a.RecoverFEMSolution(X, b, psi)
    
    # Append the result to the list (for each discrete ordinate)
    psi_mu_list.append( (mu, w, psi.Clone()) )
    
    # Optional: Save each solution to a separate file
    psi.Save("psi_mu_{:.3f}.gf".format(mu))
    mesh.Save("mesh_mu_{:.3f}.mesh".format(mu))
    
# --- Combining the Results ---
# For example, the scalar (angle-integrated) flux: ψ(x,E) = Σ_μ (w_μ * ψ_μ(x,E))
psi_scalar = psi_mu_list[0][2].Clone()  # same FE space, initialize to zero
psi_scalar.Assign(0.0)
for (mu, w, psi_mu) in psi_mu_list:
    psi_scalar.Add(w, psi_mu)

# Save the output to a file
psi_scalar.Save("psi_scalar.gf")
print("Scalar flux saved to file: psi_scalar.gf")

# Optional: To evaluate the flux going left and right separately,
# you can create separate lists for mu>0 and mu<0.
left_flux = None
right_flux = None
for (mu, w, psi_mu) in psi_mu_list:
    if mu < 0:
        if left_flux is None:
            left_flux = psi_mu.Clone()
            left_flux.Assign(0.0)
        left_flux.Add(w, psi_mu)
    else:
        if right_flux is None:
            right_flux = psi_mu.Clone()
            right_flux.Assign(0.0)
        right_flux.Add(w, psi_mu)
if left_flux:
    left_flux.Save("psi_left.gf")
    print("Left-going flux: psi_left.gf")
if right_flux:
    right_flux.Save("psi_right.gf")
    print("Right-going flux: psi_right.gf")


NameError: name 'np' is not defined

In [ ]:
import mfem
import numpy as np

# HDF5'ten okunacak verilerin burada doğrudan tanımlanması
G = 50  # enerji grup sayısı
N = 20  # uzay hücre sayısı
# Enerji gruplarının orta enerjileri (MeV) [1.00, ..., 0.02] aralığında:
E_values = np.linspace(1.0, 0.02, G)  
dE = E_values[0] - E_values[1]  # ~0.02 MeV grup genişliği
# Soğurma kesiti sigma(E) [1/cm] ve durdurma gücü S(E) [MeV cm^2/g] verileri:
sigma = np.array([2.27418E-04, 2.45855E-04, 2.46269E-04, 2.50741E-04, 2.57984E-04,
                  2.57151E-04, 2.79860E-04, 2.89205E-04, 2.81906E-04, 2.96651E-04,
                  2.99901E-04, 3.10618E-04, 3.15408E-04, 3.39896E-04, 2.97606E-04,
                  3.07739E-04, 2.95290E-04, 3.16050E-04, 2.86846E-04, 2.88778E-04,
                  3.03885E-04, 3.23915E-04, 3.39852E-04, 3.21681E-04, 3.20325E-04,
                  3.30874E-04, 3.46433E-04, 3.10682E-04, 3.37842E-04, 3.72998E-04,
                  3.86048E-04, 4.66812E-04, 5.09680E-04, 5.92211E-04, 6.61295E-04,
                  7.53455E-04, 8.95965E-04, 1.04145E-03, 1.28112E-03, 1.55454E-03,
                  1.91991E-03, 2.50250E-03, 3.38605E-03, 4.60680E-03, 6.57085E-03,
                  9.82564E-03, 1.69755E-02, 3.47682E-02, 9.96159E-02, 0.00000E+00])
S = np.array([3.11655E+00, 3.12384E+00, 3.13157E+00, 3.13944E+00, 3.14808E+00,
              3.15685E+00, 3.16580E+00, 3.17507E+00, 3.18464E+00, 3.19454E+00,
              3.20478E+00, 3.21538E+00, 3.22636E+00, 3.23774E+00, 3.24953E+00,
              3.26177E+00, 3.27449E+00, 3.28773E+00, 3.30152E+00, 3.31591E+00,
              3.33096E+00, 3.34672E+00, 3.36325E+00, 3.38063E+00, 3.39894E+00,
              3.41827E+00, 3.43873E+00, 3.46043E+00, 3.48353E+00, 3.50816E+00,
              3.53452E+00, 3.56280E+00, 3.59324E+00, 3.62609E+00, 3.66166E+00,
              3.70028E+00, 3.74236E+00, 3.78838E+00, 3.83890E+00, 3.89459E+00,
              3.95627E+00, 4.02496E+00, 4.10192E+00, 4.18879E+00, 4.28771E+00,
              4.40159E+00, 4.53454E+00, 4.69260E+00, 4.88626E+00, 5.13440E+00])

# 1B uniform mesh oluşturulması (0-0.3 arası, 20 eleman)
mesh = mfem.Mesh.MakeCartesian1D(N, 0.3)
mesh.SetCurvature(1, False)  # geometrik olarak doğrusal

# DG finite element space (L2, parçalı sabit)
fec = mfem.L2_FECollection(order=0, dim=mesh.Dimension())
fes = mfem.FiniteElementSpace(mesh, fec)

# Çözüm ve kaynak için grid fonksiyonlarının hazırlanması
phi_prev = mfem.GridFunction(fes)  # bir önceki grubun akısı
phi_prev.Assign(0.0)  # başlangıçta boş (g=0 için harici kaynak kullanılacak)

# Sınır koşulu: x=0 giriş akısı = 1.0 (g=0 grubu için)
inflow_val = 1.0
inflow_coeff = mfem.ConstantCoefficient(inflow_val)
# Taşıma hız vektörü (1D yönde)
velocity = mfem.VectorConstantCoefficient([1.0])  # (mu = 1.0)
alpha = -1.0  # upwind için negatif işaret

# Sonuçları depolayacak liste
phi_solutions = []

# Enerji grupları üzerinden iterasyon
for g in range(G):
    # Bilinear form tanımla (her grupta sıfırdan oluşturuyoruz)
    a = mfem.BilinearForm(fes)
    # Konvektif DG terimleri (iç yüzey ve sınır)
    a.AddDomainIntegrator(mfem.ConvectionIntegrator(velocity, alpha))
    a.AddInteriorFaceIntegrator(mfem.NonconservativeDGTraceIntegrator(velocity, alpha))
    a.AddBdrFaceIntegrator(mfem.NonconservativeDGTraceIntegrator(velocity, alpha))
    # Reaksiyon terimi: (sigma_g + S_g/ΔE)
    reaction_coeff = mfem.ConstantCoefficient(sigma[g] + (S[g] / dE))
    a.AddDomainIntegrator(mfem.MassIntegrator(reaction_coeff))
    a.Assemble()  # lokal integrallerin toplanması
    a.Finalize()  # matris oluşturulur

    # Sağ taraf (linear form) tanımla
    b = mfem.LinearForm(fes)
    # Sınırdan gelen akı: sadece g=0 için inflow=1, diğerleri için 0 (inflow_coeff güncellenebilir)
    if g == 0:
        inflow_coeff = mfem.ConstantCoefficient(1.0)
    else:
        inflow_coeff = mfem.ConstantCoefficient(0.0)
    b.AddBdrFaceIntegrator(mfem.BoundaryFlowIntegrator(inflow_coeff, velocity, alpha))
    b.Assemble()  # şimdilik sınır katkısı monte edilsin

    # Önceki grubun kaynak katkısı (g>0 için)
    if g > 0:
        # phi_prev halihazırda önceki grubun çözümünü içeriyor
        # Her eleman için phi_prev değerini al ve linear form vektörüne ekle
        elem_vol = 0.3 / N  # uniform hücre hacmi (1B uzunluğu)
        for elem in range(N):
            # L2 order0'da her elementin tek bir dof'u var
            prev_val = phi_prev[elem]  # önceki grup akısı (parçalı sabit)
            # Kaynak terimi: S_{g-1}/dE * phi_prev
            source_val = (S[g-1] / dE) * prev_val
            # Linear form'da ilgili dof'a ekle (test fonksiyonu 1 olduğundan hacim çarp)
            b[elem] += source_val * elem_vol

    # Şimdi lineer sistemi çözelim: A * phi_g = b
    A = a.SpMat()  # assembled SparseMatrix
    # GMRES solver kur
    solver = mfem.GMRESSolver()
    solver.SetRelTol(1e-9)
    solver.SetAbsTol(1e-12)
    solver.SetMaxIter(500)
    solver.SetPrintLevel(0)  # iterasyon bilgisi bastırma (1 yapılarak görülebilir)
    # Block ILU preconditioner (blok boyutu = her elementteki dof sayısı = 1 burada)
    prec = mfem.BlockILU(1, mfem.BlockILU.REORDER_NATURAL)  
    prec.SetMatrix(A)
    solver.SetPreconditioner(prec)
    solver.SetOperator(A)
    # Çözümü hesapla
    phi_g = mfem.GridFunction(fes)
    phi_g.Assign(0.0)  # başlangıç tahmini (sıfır)
    solver.Mult(b, phi_g)  # çözücü b'yi alıp phi_g'yi doldurur
    # Çözüm sonucunu kaydet
    phi_solutions.append(phi_g)
    # Bu çözümü bir sonraki iterasyon için phi_prev'e aktar
    phi_prev.Assign(phi_g)
    # (Opsiyonel) GMRES iterasyon sayısı ve son kalıntı bilgisi alınabilir:
    # print("GMRES iters:", solver.GetNumIterations(), " final norm:", solver.GetFinalNorm())
    
# Tüm gruplar çözüldü, uzaydaki toplam akı ve doz profili hesapla
phi_total = np.zeros(N)
for g in range(G):
    # phi_solutions[g] bir GridFunction, eleman değerlerini toplayalım
    for elem in range(N):
        phi_total[elem] += phi_solutions[g][elem]
# Doz hesabı (basit normalize akı)
dose = phi_total / phi_total[0]
print("x merkezleri:", np.linspace(0.3/N/2, 0.3 - 0.3/N/2, N))
print("Toplam akı phi(x):", phi_total)
print("Normalize doz D(x):", dose)


TypeError: new_L2_FECollection() missing required argument 'p' (pos 1)

In [1]:
import numpy as np
import mfem.ser as mfem

# Problem parametreleri
L = 0.3             # Çubuk uzunluğu (metre)
n_elements = 50     # Mesh bölge sayısı (uniform bölünme)
poly_order = 1      # DG polinom derecesi (örn. 1: lineer eleman)
N_angles = 4        # Açısal quadrature noktası sayısı (Gauss-Legendre)

# 1D Mesh oluşturulması (0--L aralığını n_elements adet segmente böler)
mesh = mfem.Mesh(n_elements, L, "SEGMENT")  # 1D segment mesh [0,L]
dim = mesh.Dimension()  # dim = 1

# DG sonlu eleman uzayı tanımı (kesintili Galerkin, polinom derecesi poly_order)
fec = mfem.DG_FECollection(poly_order, dim)  # veya mfem.L2_FECollection da kullanılabilir
fes = mfem.FiniteElementSpace(mesh, fec)

# Gauss–Legendre quadrature ile açısal yönlerin (mu) ve ağırlıkların hesaplanması
mu_vals, w_vals = np.polynomial.legendre.leggauss(N_angles)  # [-1,1] aralığında
# Not: Gauss-Legendre N_angles tek ise mu=0 bir nokta olur; burada N_angles tercihen çift alınmalı.

# Çözüm sonuçlarını saklamak için liste
angle_solutions = []  # (mu, GridFunction) çiftleri

# Her bir açısal yöndeki denklemin çözümü
for mu in mu_vals:
    # Mu = 0 (tam dik açı) durumunda ilerleme olmadığı için atlanabilir
    if abs(mu) < 1e-12:
        continue

    # MFEM vector coefficient olarak sabit hız vektörü tanımla (1D: [mu])
    vel = mfem.Vector([mu])  # 1D hız vektörü
    vel_coeff = mfem.VectorConstantCoefficient(vel)

    # Sınır koşulu (inflow) için fonksiyon tanımı:
    # x=0 ve mu>0 ise 1.0, diğer durumlarda 0.0
    def inflow_func(x):
        # x bir numpy array veya mfem.Vector olarak gelir
        if x[0] < 1e-12 and mu > 0:
            return 1.0
        else:
            return 0.0
    inflow_coeff = mfem.FunctionCoefficient(inflow_func)

    # Taşıma operatörü için BilinearForm (K) ve sağ taraf için LinearForm (b) tanımla
    K = mfem.BilinearForm(fes)
    b = mfem.LinearForm(fes)

    alpha = -1.0  # upwind için kullanılan katsayı (MFEM örnek 9'da -1 seçiliyor)
    # Eleman içi adveksiyon integratörü
    K.AddDomainIntegrator(mfem.ConvectionIntegrator(vel_coeff, alpha))
    # İç yüzeyler için upwind flux integratörü
    K.AddInteriorFaceIntegrator(mfem.NonconservativeDGTraceIntegrator(vel_coeff, alpha))
    # Dış sınır yüzeyleri için upwind flux integratörü
    K.AddBdrFaceIntegrator(mfem.NonconservativeDGTraceIntegrator(vel_coeff, alpha))
    # Dış sınırdan gelen akı (inflow) için linear form integratörü
    b.AddBdrFaceIntegrator(mfem.BoundaryFlowIntegrator(inflow_coeff, vel_coeff, alpha))

    # Matris ve vektörlerin assemble edilmesi
    K.Assemble()     # bileşenlerin toplanması (sparse matris oluşturma hazırlığı)
    b.Assemble()     # sağ taraf vektörünün oluşturulması
    K.Finalize()     # sparse matris oluştur (MFEM içi)

    # Çözüm vektörünü (GridFunction) başlat ve lineer sistemi oluştur
    phi = mfem.GridFunction(fes)
    phi.Assign(0.0)  # başlangıç tahmini sıfır
    ess_tdof_list = mfem.intArray()  # DG'de zorunlu sınır DOF yok, boş bırakılır
    A = mfem.OperatorPtr()  # operatör tutucu
    X = mfem.Vector()
    B = mfem.Vector()
    # Lineer sistemi formüle et (A * X = B)
    K.FormLinearSystem(ess_tdof_list, phi, b, A, X, B)

    # GMRES çözücüsünü ve öncüllendiriciyi hazırla
    # Not: A OperatorPtr tipinde; sparse matris elde etmek için aşağıdaki satır kullanılabilir:
    A_mat = mfem.OperatorHandle2SparseMatrix(A)  
    # Gauss-Seidel yumuşatıcı (öncülleyici)
    M = mfem.GSSmoother(A_mat)
    # GMRES iteratif çözücü
    solver = mfem.GMRESSolver()
    solver.SetOperator(A_mat)
    solver.SetPreconditioner(M)
    solver.SetRelTol(1e-12)   # göreli tolerans
    solver.SetAbsTol(1e-12)   # mutlak tolerans
    solver.SetMaxIter(500)
    solver.SetKDim(30)        # yeniden başlatma değeri (krylov boyutu)
    solver.SetPrintLevel(1)   # iterasyon bilgisi (0: sessiz, 1: özet)
    # GMRES iterasyonunu çalıştır
    solver.Mult(B, X)

    # Çözülen X vektörünü sonlu eleman çözümüne geri taşı (phi)
    K.RecoverFEMSolution(X, b, phi)
    angle_solutions.append((mu, phi))

    # Çözülen açısal akıyı GLVis formatında kaydet
    out_fname = f"sol_mu{mu:.3f}.gf"
    phi.Save(out_fname)
    print(f"mu = {mu:.3f} için çözüm kaydedildi -> {out_fname}")

# Mesh dosyasını da kaydet (GLVis görselleştirme için)
mesh.Print("mesh.mesh")
print("Mesh dosyası kaydedildi -> mesh.mesh")

# Matplotlib ile sonuçların görselleştirilmesi
import matplotlib.pyplot as plt
plt.figure()
for mu, phi in angle_solutions:
    # Her elemanın orta noktasında çözüm değerini örnekleyelim
    num_elements = mesh.GetNE()
    x_coords = []
    phi_vals = []
    for elem in range(num_elements):
        trans = mesh.GetElementTransformation(elem)
        ip = mfem.IntegrationPoint()
        ip.x = 0.0  # referans elementte merkez (xi=0)
        phys_coord = trans.Transform(ip)        # fiziksel koordinata dönüştür
        val = phi.GetValue(elem, ip)           # o noktadaki φ değerini al
        x_coords.append(phys_coord[0])
        phi_vals.append(val)
    plt.plot(x_coords, phi_vals, label=f"mu = {mu:.3f}")
plt.xlabel("x (m)")
plt.ylabel("Açısal Akı φ(x, μ)")
plt.title("CSDA Taşıma Denklemi Çözümü (Upwind DG)")
plt.legend()
plt.grid(True)
plt.show()


TypeError: Wrong number or type of arguments for overloaded function 'new_Mesh'.
  Possible C/C++ prototypes are:
    mfem::Mesh::Mesh()
    mfem::Mesh::Mesh(mfem::Mesh const &,bool)
    mfem::Mesh::Mesh(mfem::Mesh &&)
    mfem::Mesh::Mesh(mfem::real_t *,int,int *,mfem::Geometry::Type,int *,int,int *,mfem::Geometry::Type,int *,int,int,int)
    mfem::Mesh::Mesh(int,int,int,int,int)
    mfem::Mesh::Mesh(std::string const &,int,int,bool)
    mfem::Mesh::Mesh(std::istream &,int,int,bool)
    mfem::Mesh::Mesh(mfem::Mesh *[],int)
    mfem::Mesh::Mesh(int,int,int,mfem::Element::Type,bool,mfem::real_t,mfem::real_t,mfem::real_t,bool)
    mfem::Mesh::Mesh(int,int,mfem::Element::Type,bool,mfem::real_t,mfem::real_t,bool)
    mfem::Mesh::Mesh(int,mfem::real_t)
    mfem::Mesh::Mesh(mfem::Mesh *,int,int)
    mfem::Mesh::Mesh(int,int,int,char const *,bool,double,double,double,bool)
    mfem::Mesh::Mesh(int,int,char const *,bool,double,double,bool)
